# Работа со JSON формат во Python

<br>

Вежби, Веб сервиси и XML, 15.03.2017

доц. д-р. Светлана Кордумова Трајанова

<br>


****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
+ data source: http://jsonstudio.com/resources/
****


Unstructured data is often stored in a format called JavaScript Object Notation (JSON). JSON is a way to encode data structures like lists and dictionaries to strings that ensures that they are easily readable by machines. Even though JSON starts with the word Javascript, it's actually just a format, and can be read by any language.

Python has great JSON support, with the json library. We can both convert lists and dictionaries to JSON, and convert strings to lists and dictionaries. JSON data looks much like a dictionary would in Python, with keys and values stored.

We'll explore the json library as well as working with JSON files using Pandas.

In [ ]:
import pandas as pd

## imports for Python, Pandas

In [ ]:
import json
from pandas.io.json import json_normalize

## JSON example, with string

+ demonstrates creation of normalized dataframes (tables) from nested json string
+ source: http://pandas.pydata.org/pandas-docs/stable/io.html#normalization

In [ ]:
# define json string
data = [{'state': 'Florida', 
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]

In [ ]:
# use normalization to create tables from nested element
json_normalize(data, 'counties')

In [ ]:
# further populate tables created from nested element
json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

****
## JSON example, with file

+ demonstrates reading in a json file as a string and as a table
+ uses small sample file containing data about projects funded by the World Bank 
+ data source: http://jsonstudio.com/resources/

In [ ]:
# load json as string
json.load((open('data/world_bank_projects_less.json')))

In [ ]:
# load as Pandas dataframe
sample_json_df = pd.read_json('data/world_bank_projects_less.json')
sample_json_df

****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. Print the top *lendprojectcost* with the *theme* and the *project abstract* if available
4. Print the top *totalamt* value with the *impagency*, *countryshortname* and the *project_name* if available (**ASSIGNMENT**)

In [ ]:
# load as Pandas dataframe
exercise_json_df = pd.read_json('data/world_bank_projects.json')
exercise_json_df

In [ ]:
#Answer to question 1. 
#The method value_counts Returns object containing counts of unique values. 
#The resulting object will be in descending order so that the first element is the most frequently-occurring element. 
#Excludes NA values by default. Source https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.value_counts.html
exercise_json_df.countryshortname.value_counts().head(10)

In [ ]:
#Answer to question 2 

In [ ]:
code_df = exercise_json_df[['mjtheme_namecode']]
#note that this makes a new dataframe of just this column, but it is still contains a list of dictionaries for each row
#DataFrame.iloc is purely integer-location based indexing for selection by position.
#.iloc[] is primarily integer position based (from 0 to length-1 of the axis), but may also be used with a boolean array.
#Source https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.iloc.html
type(code_df.iloc[0][0]), code_df.iloc[0][0]


In [ ]:
code_df.iloc[1][0]

In [ ]:
code_df.iloc[2][0]

In [ ]:
for entry in code_df.iloc[0][0]: #to access the dictionary elements, this For loop goes through the list and finds each element
    for n in entry:
        print(entry[n])

In [ ]:
code_num = [] #creates empty lists to append each entry to
code_name = []
for i in range(len(code_df)): #goes through each row of the dataframe
    for entry in code_df.iloc[i][0]: #accesses the list as in the cell above
        for n in entry:
            l = [entry[n]] #turns the element into a list for appending
            if n == 'code':#sorts each element into the appropriate list
                code_num = code_num + l 
            else:
                code_name = code_name + l
#check out the first 10 elements
code_num[1:10], code_name[1:10]

In [ ]:
from collections import Counter
# Returns all unique items and their counts.
# A Counter is a dict subclass for counting hashable objects. 
# It is an unordered collection where elements are stored as dictionary keys and their counts are stored as dictionary values. 
# Counts are allowed to be any integer value including zero or negative counts.
# Source: https://docs.python.org/2/library/collections.html#collections.Counter
code_num = Counter(code_num)
code_name = Counter(code_name)
# most_common(n) returns a list of the n most common elements and their counts from the most common to the least.
# most_common(1) would return the highest occurring item.
# Source: https://docs.python.org/2/library/collections.html
num = code_num.most_common(10)
theme = code_name.most_common(10)  # num and theme are now lists of tuples including the counts


In [ ]:
#since I don't care about the number of occurances per se, this pulls out simply the list of codes and names, excluding count

for i in range(10):
    num[i] = num[i][0]
    theme[i] = theme[i][0]
num, theme

In [ ]:
import numpy as np

sample_json_df = pd.read_json('data/world_bank_projects.json')
lendprojectcost_list = []
theme_list = []
project_abstract_list = []

# DataFrame.itertuples(index=True, name='Pandas')[source]
# Iterate over DataFrame rows as namedtuples, with index value as first element of the tuple.
# It preserve dtypes while iterating over the rows
# Source: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.itertuples.html#pandas.DataFrame.itertuples
for row in sample_json_df.itertuples():
    lendprojectcost_list.append(row.lendprojectcost)
    theme_list.append(row.mjtheme)
    if type(row.project_abstract) is type({}):
        project_abstract_list.append(row.project_abstract.get("cdata"))
    else:
        project_abstract_list.append("No abstract available")
    
lendprojectcost_list = np.array(lendprojectcost_list)
theme_list = np.array(theme_list)
project_abstract_list = np.array(project_abstract_list)

sorted_indexes = np.argsort(lendprojectcost_list)

lendprojectcost_list = lendprojectcost_list[sorted_indexes]
theme_list = theme_list[sorted_indexes]
project_abstract_list = project_abstract_list[sorted_indexes]

# Since np.argsort gives the sorted indeces only in ascending order, we will take the last element
n=len(lendprojectcost_list)
print(lendprojectcost_list[n-1], theme_list[n-1], project_abstract_list[n-1])
